### reference
https://partner.cloudskillsboost.google/course_templates/948/video/485086

In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform umap-learn tqdm pypdf

In [ ]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part as GenerativeModelPart,
    HarmBlockThreshold,
    HarmCategory,
)


In [ ]:
import pandas as pd

data=pd.read_csv('testdata.csv',encoding='latin1')
data

In [ ]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json
from google.cloud import bigquery 
import os 
import gcsfs
import pandas as pd

def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)      

In [ ]:
request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
rf= open(request_file.name, "a") 

for idx,row in data.iterrows():
    data_items=[ json.dumps( {  "_id": idx,
                                "title":row['title'],
                                 "text":row['content']                                          
                               }
                               )  +"\n"
                ]

    rf.writelines(data_items)
    rf.flush()
upload_file(rf,dest_bucket_name='vlt_search_and_contet_output',request_file_folder='finetunning',request_file_prefix='fine_tuning_intelia_sample', version=0, request_file_post_fix ='0')


In [ ]:
def generate_queries(model="gemini-1.5-pro", title="", text=""):

        metric_prompt = (f'You are an expert who is tasked with extracting some questions from a given article.\n'
        f'Think throughly when reading this article including both title and content. Think of questions may people may ask to know more about a data consultancy called Intelia.\n'
        f'Suggest 1 to 2 short questions that are important to ask when people want to explore who Intelia is and what it does.\n'
        f' Also suggest 1-2 labels or short expressions that are related to this articleor highlight them.\n'
        f' These expressions or questions should be optimized for fine tuning a text embedding model.\n'    
        f'Output each response on a separate line divided by a newline.\n'
        f'Here is the article:\n'
        f'"title": {title}\n'
        f'"content":{text}')

        # set evaluation metric schema
        metric_response_schema = {
            "type": "OBJECT",
            "properties": {
                "question_expression": {
                    "type": "ARRAY",
                    "items": {
                        "type": "OBJECT",
                        "properties": {
                            "query_expression": {"type": "STRING"},
                            "answer": {"type": "STRING"}
                        },
                        "required": ["query_expression","answer"]
                    }
                } 
            },
            "required": ["question_expression"],
        }
    
      
        #define a generative model as an autorator
  
  
        autorater = GenerativeModel(
            model,
            generation_config=GenerationConfig(
                response_mime_type="application/json",
                response_schema=metric_response_schema,
            ),
            safety_settings={
                HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            },
        )



      
    
        #generate the rating metrics as per requested measures and metric in the prompt
        response = autorater.generate_content([metric_prompt])

        response_json = {}

        if response.candidates and len(response.candidates) > 0:
            candidate = response.candidates[0]
            if (
                candidate.content
                and candidate.content.parts
                and len(candidate.content.parts) > 0
            ):
                part = candidate.content.parts[0]
                if part.text:
                    response_json = json.loads(part.text)

        return response_json


In [ ]:
def get_autorater_response(llm_model: str="gemini-1.5-pro", title="", text="", query="") -> dict:
        
        """Extract evaluation metric on a AI-generated content using a AI-as-judge approach
        
        Args:
        list metric_prompt: the input metric prompt parameters
        str llm_model: evaluation model

        Returns:
        dict response_json: the evaluated metric in json format
        """
            
        # set evaluation metric schema
        metric_response_schema = {
            "type": "OBJECT",
            "properties": {
                "score": {"type": "NUMBER"},
                "explanation": {"type": "STRING"},
            },
            "required": ["score", "explanation"],
        }     
        #define a generative model as an autorator
        autorater = GenerativeModel(
            llm_model,
            generation_config=GenerationConfig(
                response_mime_type="application/json",
                response_schema=metric_response_schema,
            ),
            safety_settings={
                HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            },
        )

        metric_prompt= f"""
                        # Instruction 
                        You are an expert evaluator. Your task is to evaluate the amount of the relevancy between a given question and a given article. You should first read the question carefully for analyzing the task, then read the article. Then evaluate the how relevant the given question is to the given article based on the Criteria provided in the Evaluation section below. You will assign the response a rating following the Rating Rubric and Evaluation Steps. Only choose ratings from the Rating Rubric. 
                        # Evaluation
                        ## Metric Definition 
                        You will be assessing Relevancy, which measures the ability to find a clear and thorough answer to the given question within given artcile.
                        ## Criteria 
                        Relevancy: Measures the ability to find a clear and thorough answer to the given question within given artcile
                        - Category - Detailed Description Of Events And Conversations - Brands, Company Names, and Logos -
                        Key Locations And Scenes - Key Themes - People Appearing And Mentioned Thi
                        s AI-generated responses will be used for data retrieval. So, it has to be able to capture all the details of a scene. 
                        ## Rating Rubric 
                         0: No relevancy. The document has no connection to the question. It lacks relevant keywords, topics, or context. 
                         1: Low Relevance. The document contains some keywords but does not provide meaningful information related to the question. The context is weak or unrelated. \n
                         2: Moderate Relevance. The document partially addresses the question but lacks completeness, clarity, or depth. It may require inference to extract an answer.\n
                         3	High Relevance. The document directly and clearly answers the question with specific, well-aligned information. Strong contextual alignment.\n
                        
                        ## Evaluation Steps 
                        STEP 1: Assess question: Carefully read the question to underestand what it is asking. 
                        STEP 2: Readt the artcile including title and content
                        STEP 3: Evaluate relevancy: Evaluate how much the article could be relevant to the question and if the answer can be find in this article. 
                        STEP 4: Determine relevancy score: Based on the previous steps, assign a relevancy score using the
                        0-3 rubric.

                        Here is the question and article:
                        ## Question:\n
                        {query}\n                      
                        ## Article:\n                    
                        "title": 
                                {title}\n
                        "content":
                                 {text}"""

   
        #generate the rating metrics as per requested measures and metric in the prompt
        response = autorater.generate_content([metric_prompt])

        response_json = {}

        if response.candidates and len(response.candidates) > 0:
            candidate = response.candidates[0]
            if (
                candidate.content
                and candidate.content.parts
                and len(candidate.content.parts) > 0
            ):
                part = candidate.content.parts[0]
                if part.text:
                    response_json = json.loads(part.text)

        return response_json

In [ ]:
generated_queries=[]
request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
rf= open(request_file.name, "a") 
all_queries=[]
qid=0
for idx,row in data.iterrows():
    queries=generate_queries(title= row['title'],text= row['content']) 
    tmp=queries.get("question_expression", "")
    for exp in tmp:          
          all_queries.append(['doc'+str(idx),'q'+str(qid),exp['query_expression'],exp['answer']])
          qid=qid+1
    
    print(all_queries)
    

In [ ]:
# Create DataFrame with column names
df = pd.DataFrame(all_queries, columns=[ 'corpus-id' , 'query-id' ,'query','answer'])
df.to_csv('all_queries.csv')

In [ ]:
request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
rf= open(request_file.name, "a") 
q1=["What is Intelia's core philosophy and how does it impact their approach to customer success?",  'How does Intelia balance its focus on customer delight with the well-being of its employees?',  'What specific actions does Intelia take to foster a passionate and enjoyable work environment?',  'How does Intelia leverage diversity and collaboration to achieve its goals?',  'What sets Intelia apart from other data consultancies in the market?']

q2=['1. **What specific types of data migrations does Intelia specialize in?**',  '2. **Does Intelia offer end-to-end data migration services, including planning, execution, and post-migration support?**',  '3. **What cloud platforms is Intelia most experienced with and certified in?**',  '4. **Can Intelia provide case studies or examples of successful data migration projects they have completed?**',  "5. **What is Intelia's pricing model for data migration services?**"]

q3=[ '1. **What specific data-driven opportunities does Intelia help its clients unlock?** ', '2. **Beyond data analysis, what other services does Intelia provide to its clients?**', "3. **Can you elaborate on Intelia's unique approach to data consultancy?**", '4. **How does Intelia tailor its services to address the specific challenges of each client?**', '5. **What are some examples of successful projects Intelia has completed for its clients?** ']

q4=['What types of businesses does Intelia typically work with?', 'What makes Intelia different from other cloud data platform providers?', "How does Snowflake's cloud-first architecture benefit businesses?", 'How can Intelia assist with planning a cloud or multi-cloud strategy? ', 'How does Intelia help clients focus on business value rather than data warehousing complexity']

q5=[  '1. What specific data consultancy services does Intelia offer?', '2. What kind of impact does Intelia make for its customers?', '3. How does Intelia foster a culture of innovation and excellence within its team?', "4. What are Intelia's plans for the future, especially in the context of the evolving AI and data landscape?", "5. What are some examples of projects or achievements that demonstrate Intelia's expertise and success? "]

q6=['1. What specific data governance services does Intelia offer to help businesses succeed with their machine learning solutions? ', '2. How does Intelia ensure compliance and facilitate collaboration when implementing data governance strategies?', '3. Can Intelia provide examples of how they have helped businesses improve their data management and avoid the need for significant changes down the line?', '4. What are the key benefits of partnering with Intelia for data governance, particularly in the context of Generative AI adoption?', '5. Does Intelia offer tailored solutions for different industries or specific data-driven challenges? ']

q7=['- What services does Intelia provide?', "- What are some of Intelia's major accomplishments in 2024?", "- What are some of Intelia's core values?", "- What are Intelia's goals for 2025?", '- What is the meaning of the hashtag "#onetribe"?']

queries=q1+q2+q3+q4+q5+q6+q7

qid=0
for query in queries:
        if query!="":
            generated_queries=[ json.dumps( {  "_id": qid,  
                                               "text":query                                      
                                         }
                                       )  +"\n"
                        ]
        
            rf.writelines(generated_queries)
            rf.flush()
            qid +=1

upload_file(rf,dest_bucket_name='vlt_search_and_contet_output',request_file_folder='finetunning',request_file_prefix='fine_tuning_query_sample', version=0, request_file_post_fix ='0')

    

In [ ]:
queries

In [ ]:
queries=[{"_id": 0, "text": "What is Intelia's core philosophy and how does it impact their approach to customer success?"},
{"_id": 1, "text": "How does Intelia balance its focus on customer delight with the well-being of its employees?"},
{"_id": 2, "text": "What specific actions does Intelia take to foster a passionate and enjoyable work environment?"},
{"_id": 3, "text": "How does Intelia leverage diversity and collaboration to achieve its goals?"},
{"_id": 4, "text": "What sets Intelia apart from other data consultancies in the market?"},
{"_id": 5, "text": "1. **What specific types of data migrations does Intelia specialize in?**"},
{"_id": 6, "text": "2. **Does Intelia offer end-to-end data migration services, including planning, execution, and post-migration support?**"},
{"_id": 7, "text": "3. **What cloud platforms is Intelia most experienced with and certified in?**"},
{"_id": 8, "text": "4. **Can Intelia provide case studies or examples of successful data migration projects they have completed?**"},
{"_id": 9, "text": "5. **What is Intelia's pricing model for data migration services?**"},
{"_id": 10, "text": "1. **What specific data-driven opportunities does Intelia help its clients unlock?** "},
{"_id": 11, "text": "2. **Beyond data analysis, what other services does Intelia provide to its clients?**"},
{"_id": 12, "text": "3. **Can you elaborate on Intelia's unique approach to data consultancy?**"},
{"_id": 13, "text": "4. **How does Intelia tailor its services to address the specific challenges of each client?**"},
{"_id": 14, "text": "5. **What are some examples of successful projects Intelia has completed for its clients?** "},
{"_id": 15, "text": "What types of businesses does Intelia typically work with?"},
{"_id": 16, "text": "What makes Intelia different from other cloud data platform providers?"},
{"_id": 17, "text": "How does Snowflake's cloud-first architecture benefit businesses?"},
{"_id": 18, "text": "How can Intelia assist with planning a cloud or multi-cloud strategy? "},
{"_id": 19, "text": "How does Intelia help clients focus on business value rather than data warehousing complexity"},
{"_id": 20, "text": "1. What specific data consultancy services does Intelia offer?"},
{"_id": 21, "text": "2. What kind of impact does Intelia make for its customers?"},
{"_id": 22, "text": "3. How does Intelia foster a culture of innovation and excellence within its team?"},
{"_id": 23, "text": "4. What are Intelia's plans for the future, especially in the context of the evolving AI and data landscape?"},
{"_id": 24, "text": "5. What are some examples of projects or achievements that demonstrate Intelia's expertise and success? "},
{"_id": 25, "text": "1. What specific data governance services does Intelia offer to help businesses succeed with their machine learning solutions? "},
{"_id": 26, "text": "2. How does Intelia ensure compliance and facilitate collaboration when implementing data governance strategies?"},
{"_id": 27, "text": "3. Can Intelia provide examples of how they have helped businesses improve their data management and avoid the need for significant changes down the line?"},
{"_id": 28, "text": "4. What are the key benefits of partnering with Intelia for data governance, particularly in the context of Generative AI adoption?"},
{"_id": 29, "text": "5. Does Intelia offer tailored solutions for different industries or specific data-driven challenges? "},
{"_id": 30, "text": "- What services does Intelia provide?"},
{"_id": 31, "text": "- What are some of Intelia's major accomplishments in 2024?"},
{"_id": 32, "text": "- What are some of Intelia's core values?"},
{"_id": 33, "text": "- What are Intelia's goals for 2025?"},
{"_id": 34, "text": "- What is the meaning of the hashtag \"#onetribe\"?"}
]

In [ ]:
data

In [ ]:
qid=0
relevancy=[]

import time

try:
    for query in queries:
        for idx,row in data.iterrows():
            r=get_autorater_response(title=row['title'], text=row['content'], query=query['text']) 
            relevancy.append([query['_id'],idx,r.get("score", "")])
            print(relevancy)
            time.sleep(2)
            
     
except:
    print(qid)



    
        

In [ ]:
# Create DataFrame with column names
df = pd.DataFrame(relevancy, columns=['query-id' , 'corpus-id' ,  'score'])

df.to_csv('relevancies.csv')

In [119]:
df

,query-id,corpus-id,score
0,0,0,1.0
1,0,1,0.0
2,0,2,1.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
240,34,2,0.0
241,34,3,0.0
242,34,4,2.0
243,34,5,0.0


In [ ]:
len(queries)

In [120]:
import pandas as pd
df=pd.read_csv('relevancies.csv')

In [121]:
df=df[df['score']>0]
df[['query-id' , 'corpus-id' ,  'score']]

,query-id,corpus-id,score
0,0,0,1.0
2,0,2,1.0
5,0,5,1.0
7,1,0,1.0
11,1,4,1.0
...,...,...,...
229,32,5,1.0
230,32,6,1.0
238,34,0,3.0
242,34,4,2.0


In [ ]:
df[df['query-id']==9]

In [122]:
df = df.sample(frac=1).reset_index(drop=True)

train=df.reset_index(drop=True)[['query-id' , 'corpus-id' ,  'score']]


In [123]:
# Save DataFrame to TSV file
train.to_csv('train.tsv', sep='\t', index=False)



In [124]:
import re

from google.cloud.aiplatform import initializer as aiplatform_init
from vertexai.language_models import TextEmbeddingModel


def tune_embedding_model(
    api_endpoint: str,
    base_model_name: str =  'textembedding-gecko@003',
    corpus_path: str = "gs://vlt_search_and_contet_output/finetunning/fine_tuning_intelia_sample_0_0.json",
    queries_path: str = "gs://vlt_search_and_contet_output/finetunning/fine_tuning_query_sample_0_0.json",
    train_label_path: str = "gs://vlt_search_and_contet_output/finetunning/train.tsv",
    #test_label_path: str = "gs://vlt_search_and_contet_output/finetunning/test.tsv",
):  # noqa: ANN201
    """Tune an embedding model using the specified parameters.
    Args:
        api_endpoint (str): The API endpoint for the Vertex AI service.
        base_model_name (str): The name of the base model to use for tuning.
        corpus_path (str): GCS URI of the JSONL file containing the corpus data.
        queries_path (str): GCS URI of the JSONL file containing the queries data.
        train_label_path (str): GCS URI of the TSV file containing the training labels.
        test_label_path (str): GCS URI of the TSV file containing the test labels.
    """
    match = re.search(r"^(\w+-\w+)", api_endpoint)
    location = match.group(1) if match else "us-central1"
    base_model = TextEmbeddingModel.from_pretrained(base_model_name)
    tuning_job = base_model.tune_model(
        task_type="DEFAULT",
        corpus_data=corpus_path,
        queries_data=queries_path,
        training_data=train_label_path,
        #test_data=test_label_path,
        batch_size=128,  # The batch size to use for training.
        train_steps=1000,  # The number of training steps.
        tuned_model_location=location,
        #output_dimensionality=768,  # The dimensionality of the output embeddings.
        learning_rate_multiplier=1.0,  # The multiplier for the learning rate.
    )
    return tuning_job


In [127]:
tuningjob=tune_embedding_model('')

Creating PipelineJob
PipelineJob created. Resource name: projects/494586852359/locations/us-central1/pipelineJobs/tune-text-embedding-model-20250202050250
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/494586852359/locations/us-central1/pipelineJobs/tune-text-embedding-model-20250202050250')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tune-text-embedding-model-20250202050250?project=494586852359


In [ ]:
dir(tuningjob)

In [126]:
tuningjob._status

<PipelineState.PIPELINE_STATE_RUNNING: 3>

In [135]:

import pandas as pd
import gcsfs

# GCS file path
gcs_uri = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/corpus.jsonl"

# Initialize GCS file system
fs = gcsfs.GCSFileSystem()

# Read JSONL file into DataFrame
with fs.open(gcs_uri, 'r') as f:
    df = pd.read_json(f, lines=True)

# Display DataFram
df


,_id,title,text
0,corpus/0,### Note About Forward-Looking Statements,This Annual Report on Form 10-K contains forwa...
1,corpus/1,### Note About Forward-Looking Statements,that our traffic acquisition costs (TAC) and t...
2,corpus/2,### Note About Forward-Looking Statements,come online; •our expectation that our foreign...
3,corpus/3,### Note About Forward-Looking Statements,"long-term initiatives, in particular in suppor..."
4,corpus/4,### Note About Forward-Looking Statements,"our other income (expense), net (OI&E), will f..."
...,...,...,...
318,corpus/318,### CERTIFICATION OF CHIEF EXECUTIVE OFFICER,"I, Sundar Pichai, certify that: 1.I have revie..."
319,corpus/319,### CERTIFICATION OF CHIEF EXECUTIVE OFFICER,other certifying officer and I are responsible...
320,corpus/320,### CERTIFICATION OF CHIEF EXECUTIVE OFFICER,"control over financial reporting, or caused su..."
321,corpus/321,### CERTIFICATION OF CHIEF EXECUTIVE OFFICER,over financial reporting that occurred during ...


In [133]:
import pandas as pd
import gcsfs

# GCS file path
gcs_uri = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/test.tsv"

# Initialize GCS file system
fs = gcsfs.GCSFileSystem()

# Read JSONL file into DataFrame
with fs.open(gcs_uri, 'r') as f:
    df = pd.read_csv(f, sep='\t')

# Display DataFram
df

,corpus-id,query-id,score
0,corpus/195,queries/251,1
1,corpus/225,queries/221,1
2,corpus/177,queries/177,1
3,corpus/183,queries/185,1
4,corpus/306,queries/307,1
...,...,...,...
150,corpus/85,queries/86,1
151,corpus/232,queries/221,1
152,corpus/272,queries/273,1
153,corpus/203,queries/201,1


In [129]:
import re

from google.cloud.aiplatform import initializer as aiplatform_init
from vertexai.language_models import TextEmbeddingModel


def tune_embedding_model(
    api_endpoint: str,
    base_model_name: str = "text-embedding-005",
    corpus_path: str = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/corpus.jsonl",
    queries_path: str = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/queries.jsonl",
    train_label_path: str = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/train.tsv",
    test_label_path: str = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/test.tsv",
):  # noqa: ANN201
    """Tune an embedding model using the specified parameters.
    Args:
        api_endpoint (str): The API endpoint for the Vertex AI service.
        base_model_name (str): The name of the base model to use for tuning.
        corpus_path (str): GCS URI of the JSONL file containing the corpus data.
        queries_path (str): GCS URI of the JSONL file containing the queries data.
        train_label_path (str): GCS URI of the TSV file containing the training labels.
        test_label_path (str): GCS URI of the TSV file containing the test labels.
    """
    match = re.search(r"^(\w+-\w+)", api_endpoint)
    location = match.group(1) if match else "us-central1"
    base_model = TextEmbeddingModel.from_pretrained(base_model_name)
    tuning_job = base_model.tune_model(
        task_type="DEFAULT",
        corpus_data=corpus_path,
        queries_data=queries_path,
        training_data=train_label_path,
        test_data=test_label_path,
        batch_size=128,  # The batch size to use for training.
        train_steps=1000,  # The number of training steps.
        tuned_model_location=location,
        output_dimensionality=768,  # The dimensionality of the output embeddings.
        learning_rate_multiplier=1.0,  # The multiplier for the learning rate.
    )
    return tuning_job


In [130]:
job=tune_embedding_model('')

Creating PipelineJob
PipelineJob created. Resource name: projects/494586852359/locations/us-central1/pipelineJobs/tune-text-embedding-model-20250202052109
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/494586852359/locations/us-central1/pipelineJobs/tune-text-embedding-model-20250202052109')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tune-text-embedding-model-20250202052109?project=494586852359
